In [1]:
# Etapa 1: Query SQL para tab_silver_fato_patrimonio_historico

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_contabilidade@onelake.dfs.fabric.microsoft.com/lk_contabilidade.Lakehouse/Tables/tab_silver_fato_patrimonio_historico"

icalculorazaohist = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoPatrimonial.Lakehouse/Tables/icalculorazaohist"
)
icalculorazaohist.createOrReplaceTempView("icalculorazaohist")

icalculorazao = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoPatrimonial.Lakehouse/Tables/icalculorazao"
)
icalculorazao.createOrReplaceTempView("icalculorazao")

igrupopatrimonio = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoPatrimonial.Lakehouse/Tables/igrupopatrimonio"
)
igrupopatrimonio.createOrReplaceTempView("igrupopatrimonio")

ipatrimonio = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoPatrimonial.Lakehouse/Tables/ipatrimonio"
)
ipatrimonio.createOrReplaceTempView("ipatrimonio")


tprd = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tprd"
)
tprd.createOrReplaceTempView("tprd")

tnumseriecompl = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tnumseriecompl"
)
tnumseriecompl.createOrReplaceTempView("tnumseriecompl")

tnumserie = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tnumserie"
)
tnumserie.createOrReplaceTempView("tnumserie")

tloc = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tloc"
)
tloc.createOrReplaceTempView("tloc")


df_resultado = spark.sql("""
WITH DEPRECIACAO_CALCULADA AS (
    SELECT 
        i.CODCOLIGADA,
        i.IDENTIFPRODUTO,
        i.CODFILIAL,
        ig.DESCRICAO AS DESCRICAO_CONTA,
        i.PATRIMONIO,
        i.CODPATRIMONIO,
        i.IDPATRIMONIO,
        i.DESCRICAO,
        i.DATAAQUISICAO,
        i.AQUISICAO,
        i.NUMERODOCUMENTO,
        i.VALORBASE,
        i.TIPOOCORRENCIA ,
        ih.VALORBASECORRIGIDO,
        ih.DATA,
        ih.TAXADEPRECIACAOPERIODO,
        ih.TAXADEPRECIACAOACUMULADA,
        ih.DEPRECIACAOACUMULADA,
        ih.SALDORESIDUAL
    FROM ipatrimonio i
    LEFT JOIN icalculorazaohist ih
        ON i.CODCOLIGADA = ih.CODCOLIGADA
       AND i.IDPATRIMONIO = ih.IDPATRIMONIO
    LEFT JOIN igrupopatrimonio ig
        ON i.CODCOLIGADA = ig.CODCOLIGADA
       AND CAST(i.IDGRUPOPATRIMONIO AS FLOAT) = CAST(ig.CODGRUPOPATRIMONIO AS FLOAT)
    WHERE i.CODCOLIGADA = 2
      AND i.ATIVO = '1'
    UNION
    SELECT 
        i.CODCOLIGADA,
        i.IDENTIFPRODUTO,
        i.CODFILIAL,
        ig.DESCRICAO AS DESCRICAO_CONTA,
        i.PATRIMONIO,
        i.CODPATRIMONIO,
        i.IDPATRIMONIO,
        i.DESCRICAO,
        i.DATAAQUISICAO,
        i.AQUISICAO,
        i.NUMERODOCUMENTO,
        i.VALORBASE,
        i.TIPOOCORRENCIA,
        ic.VALORBASECORRIGIDO,
        ic.DATA,
        ic.TAXADEPRECIACAOPERIODO,
        ic.TAXADEPRECIACAOACUMULADA,
        ic.DEPRECIACAOACUMULADA,
        ic.SALDORESIDUAL
    FROM ipatrimonio i
    LEFT JOIN icalculorazao ic
        ON i.CODCOLIGADA = ic.CODCOLIGADA
       AND i.IDPATRIMONIO = ic.IDPATRIMONIO
    LEFT JOIN igrupopatrimonio ig
        ON i.CODCOLIGADA = ig.CODCOLIGADA
       AND CAST(i.IDGRUPOPATRIMONIO AS FLOAT) = CAST(ig.CODGRUPOPATRIMONIO AS FLOAT)
    WHERE i.CODCOLIGADA = 2
      AND i.ATIVO = '1'
)
SELECT 
    dc.*,
    t.CODIGOPRD,
    t.NOMEFANTASIA,
    c.NUMSERIE,
    c.RN12,
    c.NRLOTE,
    s.CODLOC,
    t2.NOME   AS LOCAL_ESTOQUE,
    t2.CIDADE,
    t2.CODETD AS ESTADO,
    CASE
        WHEN dc.TIPOOCORRENCIA IN (99, 100) THEN 'SIM'
        ELSE 'NAO'
    END AS BAIXA_NUCLEUS
FROM DEPRECIACAO_CALCULADA dc
    LEFT JOIN TNUMSERIE s
        ON s.CODCOLIGADA = dc.CODCOLIGADA
       AND TRY_CAST(dc.PATRIMONIO AS DECIMAL(38,10)) = TRY_CAST(s.NUMSERIE AS DECIMAL(38,10))
    LEFT JOIN TPRD t
        ON s.CODCOLIGADA    = t.CODCOLIGADA
       AND s.IDPRD  = t.IDPRD
    LEFT JOIN TNUMSERIECOMPL c
        ON s.CODCOLIGADA = c.CODCOLIGADA
       AND s.IDPRD       = c.IDPRD
       AND TRY_CAST(dc.PATRIMONIO AS DECIMAL(38,10)) = TRY_CAST(c.NUMSERIE AS DECIMAL(38,10))
    LEFT JOIN TLOC t2
        ON dc.CODFILIAL   = t2.CODFILIAL
       AND dc.CODCOLIGADA = t2.CODCOLIGADA
       AND s.CODLOC       = t2.CODLOC
    WHERE LEFT(t.CODIGOPRD,2) <> 50   
""")

# Salva no Lakehouse de destino usando caminho físico
df_resultado.write.format("delta").mode("overwrite").save(path_destino)


StatementMeta(, e1cbd2af-2e23-4155-bad6-50fd7bf08d51, 3, Finished, Available, Finished)